# 🔗 What Are Chains in LangChain?

## **📌 Definition**
In LangChain, a **chain** is a **sequence of steps** (like prompt templates, LLMs, tools, or functions) that are **connected together** to form a complete **AI workflow**.

Think of a chain as a **pipeline** that takes an input, processes it through multiple steps, and produces a final output.

---

## **🔹 Why Use Chains?**
Chains let you:
- ✅ Combine multiple operations (e.g. prompt → LLM → output parsing)
- ✅ Reuse and modularize logic
- ✅ Build powerful workflows like **question answering, summarization, RAG, chatbots**, etc.

---

## **🔹 Common Use Cases for Chains**
| **Use Case** | **Example** |
|--------------|-------------|
| Summarization | Input text → Prompt → LLM → Summary |
| Q&A over documents | Question → Embed → Search → Prompt → LLM |
| Chatbot | User input → Memory → Prompt → LLM → Response |
| Data enrichment | Product name → Prompt → LLM → Tags, description, etc. |

---

## **🔹 Chain = Steps Connected Together**
```text
Input → [Prompt Template] → [Custom Runnbale] → [LLM] → [Parser] → Output
```
`chain = prompt | RunnableLambda | llm | parser`

---

# ⚙️ What Are Runnables in LangChain?

## 📌 Definition
**Runnables** are the **core building blocks** of LangChain's modern **LCEL (LangChain Expression Language)**.

A **Runnable** is an object that:
- Can be **invoked** with an input (`.invoke()`).
- Can be **chained** using the `|` operator.
- Can be **composed** to build flexible and powerful AI workflows.

Many LangChain components, including `LLMs`, `output parsers`, `retrievers`, and `prompt templates`, implement the Runnable interface. 
 
---

## 🔗 Why Use Runnables?

| ✅ Feature | 📖 Description |
|-----------|----------------|
| **Composable** | Combine steps like prompts, models, tools, and functions |
| **Modular** | Build reusable and testable components |
| **Flexible** | Add branching, parallelism, retry logic, validation, etc. |
| **Fast & Declarative** | Easier to read and debug than old-style chains |

---

## 🧩 Types of Runnables

| **Runnable Type** | **Purpose** |
|-------------------|-------------|
| `RunnableLambda` | Wraps a Python function |
| `RunnableSequence` | Runs steps in sequence |
| `RunnableParallel` | Runs multiple steps in parallel |
| `RunnablePassthrough` | Forwards input unchanged |
| `.assign()` | Adds computed fields |
| `.bind()` | Binds constant arguments |
| `.with_retry()` | Adds retry logic |
| `.with_types()` | Adds input/output validation |
|`.get_graph()` | Visualize chain structure |

---



In [37]:
# read api_key from file
with open('../api_keys.txt', 'r') as file:
    api_key = file.read()
    # print(api_key)

import os
os.environ["OPENAI_API_KEY"] = api_key

In [21]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

In [22]:
promt_from_template = ChatPromptTemplate.from_template("Make up a funny company name for a company that produces {product}")
promt_from_template

ChatPromptTemplate(input_variables=['product'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['product'], template='Make up a funny company name for a company that produces {product}'))])

### ☕️ Or:

In [23]:
from langchain_core.prompts import ChatPromptTemplate

prompt_from_messages = ChatPromptTemplate.from_messages([
    ("user", "Make up a funny company name for a company that produces {product}")
])

prompt_from_messages

ChatPromptTemplate(input_variables=['product'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['product'], template='Make up a funny company name for a company that produces {product}'))])

In [24]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model="gpt-3.5-turbo",
    temperature=1,
)

In [25]:
# You can also use promt_from_template
chain = prompt_from_messages | llm | StrOutputParser() 
 
chain.invoke({"product": "Computers"})

'Byte Me Computers'

## 📌 **L**ang**C**hain **E**xpression **L**anguage (LCEL)

**LCEL (LangChain Expression Language)** is a new, powerful way to build AI workflows in LangChain using **composable Runnables**.  
It replaces the older `Chain`-based APIs with a more **modular, flexible, and readable syntax**.

---

## 🧠 Key Concepts

| **Feature** | **Description** |
|-------------|-----------------|
| **Composable** | Chain together steps using the `|` operator |
| **Modular** | Each step is a `Runnable` (LLM, prompt, parser, logic, etc.) |
| **Declarative** | Define pipelines in a readable, function-like style |
| **Extensible** | Add custom logic, parallelism, retries, type validation |

---

- https://python.langchain.com/docs/concepts/lcel/

## **LCEL Syntax**
To understand LCEL syntax let's first build a simple chain in typical Python syntax.

### Using LCEL the format is different, rather than relying on Chains we simple chain together each component using the pipe operator `|`

### Let's break it down

In [26]:
chat_prompt = prompt_from_messages.invoke({"product": "Computers"})
chat_prompt

ChatPromptValue(messages=[HumanMessage(content='Make up a funny company name for a company that produces Computers')])

In [27]:
response = llm.invoke(chat_prompt)
response

AIMessage(content='Byte Me Computing Co.', response_metadata={'token_usage': {'completion_tokens': 6, 'prompt_tokens': 19, 'total_tokens': 25, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-9ee6064d-9294-4ae3-a051-49bab73809cd-0')

In [28]:
StrOutputParser().invoke(response)

'Byte Me Computing Co.'

## **How the Pipe (`|`) Operator Works**
### **🔹 Bitwise OR (`|`) for Integers**


In [29]:
(21 | 22) > 23

False

In [30]:
(21 | 22) > 21.5

True

## **In Langchain**
To really understand LCEL we can take a look at how this pipe operation works. We know it takes output from the right and feeds it to the left — but this isn't typical Python, so how is this implemented? Let's try creating our own version of this with some simple functions.

We will be using the `__or__` method within Python class objects. When we place two classes together like c`hain = class_a | class_b` the Python interpreter will check if these classes contain this `__or__` method. If they do then our `|` code will be translated into `chain = class_a.__or__(class_b)`.

That means both of these patterns will return the same thing:

```python
# object approach
chain = class_a.__or__(class_b)
chain("some input")

# pipe approach
chain = class_a | class_b
chain("some input")
```

With that in mind, we can build a `Runnable` class that consumes a function and turns it into a function that can be chained with other functions using the pipe operator `|`.

```Python
class Runnable:
    def __init__(self, func):
        self.execute = func  # Store the function

    def __or__(self, next_runnable): # other is a Runnable object
        def chained_func(*args, **kwargs):
            # Ensure the result from self.func is passed to other.func correctly
            return next_runnable.execute(self.execute(*args, **kwargs))
        return Runnable(chained_func)
    

    def invoke(self, *args, **kwargs):  # Ensure invoke method exists
        return self.execute(*args, **kwargs)
```
---


## 🧱 Breaking Down the Custom `Runnable` Class in Python

### **1. Constructor (`__init__` Method)**

```Python
class Runnable:
    def __init__(self, func):
        self.execute = func
```

- Takes a function `(func)` as an argument.
- Stores this function inside the instance (`self.execute`).
- This allows Runnable to wrap any function and call it later.

---

### **2. Overloading the `|` (Pipe) Operator**

```python
def __or__(self, next_runnable):
```

- This overloads the `|` operator, allowing chaining (`Runnable | Runnable`).
- When used (`A | B`), `B` is applied to the result of `A`.



```python
def chained_func(*args, **kwargs):
```
- Defines `chained_func`, which wraps the two functions being chained.
- `*args, **kwargs` allow passing any number of arguments.

```python
return next_runnable.execute(self.execute(*args, **kwargs))
```

- Calls `self.execute` (the first function in the chain).
- Passes its output as input to `next_runnable.execute` (the next function in the chain).
- Executes `next_runnable` immediately with the result from `self.execute`.



```python
return Runnable(chained_func)
```
- Returns a new `Runnable` instance containing `chained_func`, enabling further chaining.

--- 

### **3. Calling the Function (`invoke` Method)**

```python
def invoke(self, *args, **kwargs):  # Ensure invoke method exists
        return self.func(*args, **kwargs)

```
- Defines `invoke`, allowing instances of `Runnable` to be called like functions.
- When called, it executes the wrapped function (`self.func`).


In [31]:
class Runnable:
    def __init__(self, func):
        self.execute = func  # Store the function


    def __or__(self, next_runnable): # other is a Runnable object
        def chained_func(*args, **kwargs):
            # Ensure the result from self.func is passed to other.func correctly
            return next_runnable.execute(self.execute(*args, **kwargs))
        return Runnable(chained_func)


    def invoke(self, *args, **kwargs):  # Ensure invoke method exists
        return self.execute(*args, **kwargs)

## 👀 Example

In [34]:
# Define functions
def add_five(x):
    return x + 5


def multiply_by_two(x):
    return x * 2


# Wrap the functions with Runnable
add_five_runnable = Runnable(add_five)
multiply_by_two_runnable = Runnable(multiply_by_two)


# Chain the functions
chain = add_five_runnable.__or__(multiply_by_two_runnable)  # First add 5, then multiply by 2


# Invoke the chain
chain.invoke(3)
# Output: (3 + 5) * 2 = 16

16

### Using `__or__` directly we get the correct answer, now let's try using the pipe operator `|` to chain them together:

In [33]:
# chain the runnable functions together
chain = add_five_runnable | multiply_by_two_runnable

# invoke the chain
chain.invoke(3)

16